In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

import keras
import tensorflow as tf

from utility import sdr, models

print(tf.__version__)

2.8.0


In [1]:
class TRUNet(nn.Module):
    def __init__(self):
        super(TRUNet, self).__init__()
        self.down1 = StandardConv1d(4,64,5,2)
        self.down2 = DepthwiseSeparableConv1d(64, 128, 3, 1)
        self.down3 = DepthwiseSeparableConv1d(128, 128, 5, 2)
        self.down4 = DepthwiseSeparableConv1d(128, 128, 3, 1)
        self.down5 = DepthwiseSeparableConv1d(128, 128, 5, 2)
        self.down6 = DepthwiseSeparableConv1d(128, 128, 3, 2)
        self.FGRU = GRUBlock(128, 64, 64, bidirectional=True)
        self.TGRU = GRUBlock(64, 128, 64, bidirectional=False)
        self.up1 = FirstTrCNN(64, 64, 3, 2)
        self.up2 = TrCNN(192, 64, 5, 2)
        self.up3 = TrCNN(192, 64, 3, 1)
        self.up4 = TrCNN(192, 64, 5, 2)
        self.up5 = TrCNN(192, 64, 3, 1)
        self.up6 = LastTrCNN(128, 5, 5, 2)
  

    def forward(self, x):
        x1 = self.down1(x)
        x2 = self.down2(x1)
        x3 = self.down3(x2)
        x4 = self.down4(x3)
        x5 = self.down5(x4)
        x6 = self.down6(x5)
        x7 = x6.transpose(1,2)
        x8 = self.FGRU(x7)
        x9 = x8.transpose(1,2)
        x10 = self.TGRU(x9)
        x11 = self.up1(x10)
        x12 = self.up2(x11,x5)
        x13 = self.up3(x12,x4)
        x14 = self.up4(x13,x3)
        x15 = self.up5(x14,x2)
        x16 = self.up6(x15,x1)
        return x16


NameError: name 'nn' is not defined

In [ ]:
TRU = TRUNet()
total_params = sum(p.numel() for p in TRU.parameters())
print("total params:",total_params)
x = torch.randn(1, 4, 257)
print("input_shape:",x.shape)
y = TRU(x)
print("output_shape:",y.shape)